In [ ]:
from IPython.display import clear_output


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install bio
clear_output()

In [ ]:

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB import PDBList, PDBIO, Select
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import math
from math import sqrt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LogisticRegression

In [ ]:
from xlrd.formula import colname
df_all_complexes = pd.read_excel('drive/MyDrive/PPComplexes_pKD.xlsx', index_col=0, header=1)
df_all_complexes

,PDB code,Subset,Complex Type,Resolution,Affinity Data,pKd pKi pIC50,Release Year,Protein Name,Ligand Name,Pubmed ID,...,Exact Mass,No. of atoms,No. of bonds,Polar Surface Area,XLOGP3,open banel LogP,HB donor,HB acceptor,Rotatable bonds,Canonical SMILES
ID,,,,,,,,,,,,,,,,,,,,,
1,1a22,general,Protein-Protein,2.6,Kd=0.34nM,9.47,1998,GROWTH HORMONE RECEPTOR,G120R mutant human growth hormone (hGH),9571026.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5bxq,general,Protein-Protein,2.5,Kd=150nM,6.82,1998,Ran GTPase,Nuclear Transport Factor 2,9533885.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1a3b,general,Protein-Protein,1.8,Kd=0.3nM,9.52,1998,thrombin alpha,21-mer,8272424.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1a4y,general,Protein-Protein,2,Ki=1fM,15.00,1998,ANGIOGENIN,RIBONUCLEASE INHIBITOR,9311977.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1acb,general,Protein-Protein,2,Kd=0.2nM,9.70,1993,bovine alpha-chymotrypsin,leech (Hirudo medicinalis) protein proteinase ...,3071573.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,6um5,general,Protein-Protein,4.2,Kd=532nM,6.27,2019,CH848 10.17DT Env,HIV-1 neutralizing antibody DH270 UCA3,31806786.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2846,6umt,general,Protein-Protein,1.99,Kd=2.6nM,8.59,2019,"Programmed cell death protein 1, human PD-1 (N...","Programmed cell death 1 ligand 2, PD-L2 IgV",31727844.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2847,6uvo,general,Protein-Protein,2.9,Kd=579pM,9.24,2019,"RSV G central conserved domain, RSV G (157-197)",broadly neutralizing human monoclonal antibody...,31852779.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:
#Функция для нахождения точки пересечения вектора с плоскостью
def add_v3v3(v0, v1):
    return (
        v0[0] + v1[0],
        v0[1] + v1[1],
        v0[2] + v1[2],
    )


def sub_v3v3(v0, v1):
    return (
        v0[0] - v1[0],
        v0[1] - v1[1],
        v0[2] - v1[2],
    )


def dot_v3v3(v0, v1):
    return (
        (v0[0] * v1[0]) +
        (v0[1] * v1[1]) +
        (v0[2] * v1[2])
    )


def len_squared_v3(v0):
    return dot_v3v3(v0, v0)


def mul_v3_fl(v0, f):
    return (
        v0[0] * f,
        v0[1] * f,
        v0[2] * f,
    )

def isect_line_plane_v3_4d(p0, p1, plane, epsilon=1e-6):
    u = sub_v3v3(p1, p0)
    dot = dot_v3v3(plane, u)

    if abs(dot) > epsilon:
        # Calculate a point on the plane
        # (divide can be omitted for unit hessian-normal form).
        p_co = mul_v3_fl(plane, -plane[3] / len_squared_v3(plane))

        w = sub_v3v3(p0, p_co)
        fac = -dot_v3v3(plane, w) / dot
        u = mul_v3_fl(u, fac)
        return add_v3v3(p0, u)

    return None

#Функция для поиска точек с наибольшим расстоянием
def dist(p1, p2):
  x0 = p1[0] - p2[0]
  y0 = p1[1] - p2[1]
  z0 = p1[2] - p2[2]
  return x0 * x0 + y0 * y0 + z0 * z0


def maxDist(p):
  n = p.shape[0]
  maxm = 0
  for i in range(n):
    for j in range(i + 1, n):
      if dist(p.iloc[i], p.iloc[j]) > maxm:
        maxm = dist(p.iloc[i], p.iloc[j])
        max_1 = i
        max_2 = j

  return [max_1, max_2, sqrt(maxm)]

def plot_linear_cube(x, y, z, dx, dy, dz, color='red'):
    xx = [x, x, x+dx, x+dx, x]
    yy = [y, y+dy, y+dy, y, y]
    kwargs = {'alpha': 1, 'color': color}
    ax.plot3D(xx, yy, [z]*5, **kwargs)
    ax.plot3D(xx, yy, [z+dz]*5, **kwargs)
    ax.plot3D([x, x], [y, y], [z, z+dz], **kwargs)
    ax.plot3D([x, x], [y+dy, y+dy], [z, z+dz], **kwargs)
    ax.plot3D([x+dx, x+dx], [y+dy, y+dy], [z, z+dz], **kwargs)
    ax.plot3D([x+dx, x+dx], [y, y], [z, z+dz], **kwargs)

In [ ]:

# 0: protein 1
# 1: protein 2
# 2: hbond acceptor
# 3: hbond donor
# 4: weak hbond acceptor?
# 5: weak hbond donor
# 6: pos ionisable
# 7: neg ionisable
# 8: hydrophobe
# 9: carbonyl oxygen
# 10: carbonyl carbon
# 11: aromatic


hbond_acceptor = {'ALA': ['O', 'OXT'],
                  'ARG': ['O', 'OXT'],
                  'ASN': ['ND2', 'O', 'OD1', 'OXT'],
                  'ASP': ['OD1', 'OD2', 'O', 'OXT'],
                  'CYS': ['SG', 'O', 'OXT'],
                  'GLN': ['NE2','O', 'OE1', 'OXT'],
                  'GLU': ['OE1', 'OE2', 'O', 'OXT'],
                  'GLY': ['O', 'OXT'],
                  'HIS': ['ND1', 'NE2', 'CE1', 'CD2', 'O', 'OXT'],
                  'ILE': ['O', 'OXT'],
                  'LEU': ['O', 'OXT'],
                  'LYS': ['O', 'OXT'],
                  'MET': ['SD', 'O', 'OXT'],
                  'PHE': ['O', 'OXT'],
                  'PRO': ['O', 'OXT'],
                  'SER': ['OG', 'O', 'OXT'],
                  'THR': ['OG1', 'O', 'OXT'],
                  'TRP': ['O', 'OXT'],
                  'TYR': ['OH', 'O', 'OXT'],
                  'VAL': ['O', 'OXT']}


hbond_donor = {'ALA': ['N'],
                  'ARG': ['N', 'NE', 'NH1', 'NH2'],
                  'ASN': ['N', 'ND2', 'OD1'],
                  'ASP': ['N'],
                  'CYS': ['N', 'SG'],
                  'GLN': ['N', 'NE2', 'OE1'],
                  'GLU': ['N'],
                  'GLY': ['N'],
                  'HIS': ['N', 'ND1', 'CE1', 'NE2', 'CD2'],
                  'ILE': ['N'],
                  'LEU': ['N'],
                  'LYS': ['N', 'NZ'],
                  'MET': ['N'],
                  'PHE': ['N'],
                  'SER': ['N', 'OG'],
                  'THR': ['N', 'OG1'],
                  'TRP': ['N', 'NE1'],
                  'TYR': ['N', 'OH'],
                  'VAL': ['N']}

weak_hbond_acceptor = {'ALA': ['O', 'OXT'],
                  'ARG': ['O', 'OXT'],
                  'ASN': ['O', 'OD1', 'ND2', 'OXT'],
                  'ASP': ['OD1', 'OD2', 'O', 'OXT'],
                  'CYS': ['SG', 'O', 'OXT'],
                  'GLN': ['NE2','O', 'OE1', 'OXT'],
                  'GLU': ['OE1', 'OE2', 'O', 'OXT'],
                  'GLY': ['O', 'OXT'],
                  'HIS': ['ND1', 'NE2', 'CE1', 'CD2', 'O', 'OXT'],
                  'ILE': ['O', 'OXT'],
                  'LEU': ['O', 'OXT'],
                  'LYS': ['O', 'OXT'],
                  'MET': ['SD', 'O', 'OXT'],
                  'PHE': ['O', 'OXT'],
                  'PRO': ['O', 'OXT'],
                  'SER': ['OG', 'O', 'OXT'],
                  'THR': ['OG1', 'O', 'OXT'],
                  'TRP': ['O', 'OXT'],
                  'TYR': ['OH', 'O', 'OXT'],
                  'VAL': ['O', 'OXT']}

weak_hbond_donor = {'ALA': ['CA', 'CB'],
                  'ARG': ['CA', 'CB', 'CG', 'CD'],
                  'ASN': ['CA', 'CB'],
                  'ASP': ['CA', 'CB'],
                  'CYS': ['CA', 'CB'],
                  'GLN': ['CA', 'CB', 'CG'],
                  'GLU': ['CA', 'CB', 'CG'],
                  'GLY': ['CA'],
                  'HIS': ['CA', 'CB'],
                  'ILE': ['CA', 'CB', 'CG1', 'CD1', 'CG2'],
                  'LEU': ['CA', 'CB', 'CG', 'CD1', 'CD2'],
                  'LYS': ['CA', 'CB', 'CG', 'CD', 'CE'],
                  'MET': ['CA', 'CB', 'CG', 'CE'],
                  'PHE': ['CA', 'CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
                  'PRO': ['CA', 'CB', 'CG', 'CD'],
                  'SER': ['CA', 'CB'],
                  'THR': ['CA', 'CB', 'CG2'],
                  'TRP': ['CA', 'CB', 'CD1', 'CE3', 'CZ3', 'CH2', 'CZ2'],
                  'TYR': ['CA', 'CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
                  'VAL': ['CA', 'CB', 'CG1', 'CG2']}

pos_ionisable = {'ARG': ['NE', 'CZ', 'NH1', 'NH2'],
                  'HIS': ['CG', 'ND1', 'CE1', 'NE2', 'CD2'],
                  'LYS': ['NZ']}

neg_ionisable = {'ASP': ['OD1', 'OD2'],
                 'GLU': ['OE1', 'OE2']}

hydrophobe = {'ALA': [ 'CB'],
                  'ARG': ['CB', 'CG'],
                  'ASN': ['CB'],
                  'ASP': ['CB'],
                  'CYS': ['CB'],
                  'GLN': ['CB', 'CG'],
                  'GLU': ['CB', 'CG'],
                  'HIS': ['CB'],
                  'ILE': ['CB', 'CG1', 'CD1', 'CG2'],
                  'LEU': ['CB', 'CG', 'CD1', 'CD2'],
                  'LYS': ['CB', 'CG', 'CD'],
                  'MET': ['CB', 'CG', 'CE', 'SD'],
                  'PHE': ['CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
                  'PRO': ['CB', 'CG'],
                  'THR': ['CG2'],
                  'TRP': ['CB', 'CG', 'CD2', 'CE3', 'CZ3', 'CH2', 'CZ2'],
                  'TYR': ['CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2'],
                  'VAL': ['CB', 'CG1', 'CG2']}

carbonyl_oxygen = {'ALA': ['O'],
                  'ARG': ['O'],
                  'ASN': ['O'],
                  'ASP': ['O'],
                  'CYS': ['O'],
                  'GLN': ['O'],
                  'GLU': ['O'],
                  'GLY': ['O'],
                  'HIS': ['O'],
                  'ILE': ['O'],
                  'LEU': ['O'],
                  'LYS': ['O'],
                  'MET': ['O'],
                  'PHE': ['O'],
                  'PRO': ['O'],
                  'SER': ['O'],
                  'THR': ['O'],
                  'TRP': ['O'],
                  'TYR': ['O'],
                  'VAL': ['O']}

carbonyl_carbon = {'ALA': ['C'],
                  'ARG': ['C'],
                  'ASN': ['C'],
                  'ASP': ['C'],
                  'CYS': ['C'],
                  'GLN': ['C'],
                  'GLU': ['C'],
                  'GLY': ['C'],
                  'HIS': ['C'],
                  'ILE': ['C'],
                  'LEU': ['C'],
                  'LYS': ['C'],
                  'MET': ['C'],
                  'PHE': ['C'],
                  'PRO': ['C'],
                  'SER': ['C'],
                  'THR': ['C'],
                  'TRP': ['C'],
                  'TYR': ['C'],
                  'VAL': ['C']}

aromatic = {'HIS': ['CG', 'ND1', 'CE1', 'NE2', 'CD2'],
            'PHE': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
            'TRP': ['CG', 'CD1', 'CD2', 'NE1', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2'],
            'TYR': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ']}

In [ ]:
from math import radians, sin, cos

def rotate_point_wrt_center(point_to_be_rotated, angle, center_point = (0,0)):
        
    angle = radians(angle)
        
    xnew = cos(angle)*(point_to_be_rotated[0] - center_point[0]) - sin(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[0]
    ynew = sin(angle)*(point_to_be_rotated[0] - center_point[0]) + cos(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[1]
    
    return [xnew, ynew]

rotate_point_wrt_center(point_to_be_rotated = (0,1), angle = 0, center_point = (0, 0)) # angle is negative to indicate clock-wise rotation.

[0.0, 1.0]

In [ ]:
structures_files = pd.read_csv('drive/MyDrive/Data_files_3.csv', header=None)
structures_files

,0,1,2
0,0,1a22.npy,9.47
1,1,1acb.npy,9.70
2,2,1an1.npy,9.05
3,3,1atn.npy,9.35
4,4,1avx.npy,13.22
...,...,...,...
2649,1909,6uda.npy,8.05
2650,1910,6um5.npy,6.27
2651,1911,6uvo.npy,9.24
2652,1912,6uys.npy,5.80


In [ ]:
structures_files.iloc[2496]

0        1743
1    6plm.npy
2        7.05
Name: 2496, dtype: object

In [ ]:
#with open("drive/MyDrive/Dataset/Data_files_test.csv", mode="w", encoding='utf-8') as w_file:
#   names = ["Num", "Str_file", "pKD"]
#   file_writer = csv.DictWriter(w_file, delimiter = ",", 
#                                 lineterminator="\r", fieldnames=names)
#   file_writer.writeheader()

#with open("drive/MyDrive/Dataset/Diff_struct_new.csv", mode="w", encoding='utf-8') as w_file:
#    names = ["Num", "pdb_id"]
#    file_writer = csv.DictWriter(w_file, delimiter = ",", 
#                                 lineterminator="\r", fieldnames=names)
#    file_writer.writeheader()

In [ ]:



for str_num in range(670, 876):
  struct_id = structures_files[1][str_num][:4]
  pdbl = PDBList()
  pdbl.retrieve_pdb_file(struct_id, pdir='drive/MyDrive/Structures')
  parser = MMCIFParser()
  pdb = f'drive/MyDrive/Structures/{struct_id}.cif'
  structure = parser.get_structure({struct_id}, pdb)
  clear_output()

  Atoms_coord_df = pd.DataFrame(columns=['Chain_name', 'Chain_num', 'Residue', 'Type', 'x', 'y', 'z']) 
  chains = structure[0].get_chains()
  num_chain = 0
  for chain in chains:
    ch_id = chain.get_id()
    atoms = chain.get_atoms()
    num_chain+=1
    for atom in atoms:
      coord_atom = atom.get_coord()
      name_atom = atom.get_name()
      if name_atom[0] == 'H':
        continue
      res_name = str(atom.get_parent())[9:12]
      if atom.get_full_id()[3][0] == ' ':
        Atoms_coord_df.loc[len(Atoms_coord_df.index)] = [ch_id, num_chain-1, res_name, name_atom, coord_atom[0], coord_atom[1], coord_atom[2]]
  #Atoms_coord_df.replace('OXT', 'O', inplace=True)

  #Получение центров масс для белков
  center_1=[]
  x1=0
  y1=0
  z1=0
  k1=0
  center_2=[]
  x2=0
  y2=0
  z2=0
  k2=0
  for j in range(Atoms_coord_df.shape[0]):
    if Atoms_coord_df['Chain_num'][j]==0:
      x1+=Atoms_coord_df['x'][j]
      y1+=Atoms_coord_df['y'][j]
      z1+=Atoms_coord_df['z'][j]
      k1+=1
    elif Atoms_coord_df['Chain_num'][j]==1:
      x2+=Atoms_coord_df['x'][j]
      y2+=Atoms_coord_df['y'][j]
      z2+=Atoms_coord_df['z'][j]
      k2+=1
  center_1=[round(x1/k1), round(y1/k1), round(z1/k1)]
  center_2=[round(x2/k2), round(y2/k2), round(z2/k2)]



  logreg_clf = LogisticRegression()
  logreg_clf.fit(Atoms_coord_df[['x', 'y', 'z']].values, Atoms_coord_df['Chain_num'].astype('int'))

  # plane_bind - список коэффициентов [A,B,C,D] в уравнении разделяющей плоскости вида A*x+B*y+C*z+D=0, то есть 
  plane_bind = list(logreg_clf.coef_[0])
  plane_bind.append(logreg_clf.intercept_[0])

  #dot_is - координаты точки пересечения, а следовательно и центра ячейки
  dot_is = isect_line_plane_v3_4d(center_1, center_2, plane_bind)
  x0, y0, z0 = dot_is[0], dot_is[1], dot_is[2]
  x1, y1, z1 = center_1[0], center_1[1], center_1[2]
  x2, y2, z2 = center_2[0], center_2[1], center_2[2]
  n_v = [(x1 - x0), (y1 - y0), (z1 - z0)]

  plane_3_1=[plane_bind[0], plane_bind[1], plane_bind[2], plane_bind[3]+3]
  plane_3_2=[plane_bind[0], plane_bind[1], plane_bind[2], plane_bind[3]-3]

  #Выбираем атомы, которые примерно лежат на этой плоскости, подставляем их координаты в уравнение плоскости, чтобы понять насколько близко он к ней расположены
  Atoms_3A_1 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
  for i in range(Atoms_coord_df.shape[0]):
    x = Atoms_coord_df['x'][i]
    y = Atoms_coord_df['y'][i]
    z = Atoms_coord_df['z'][i]
    d = plane_3_1[0]*x + plane_3_1[1]*y + plane_3_1[2]*z + plane_3_1[3]
    if abs(d)<0.3:
      Atoms_3A_1.loc[len(Atoms_3A_1.index)] = [i, x, y, z, d]

  Atoms_3A_2 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
  for i in range(Atoms_coord_df.shape[0]):
    x = Atoms_coord_df['x'][i]
    y = Atoms_coord_df['y'][i]
    z = Atoms_coord_df['z'][i]
    d = plane_3_2[0]*x + plane_3_2[1]*y + plane_3_2[2]*z + plane_3_2[3]
    if abs(d)<0.3:
      Atoms_3A_2.loc[len(Atoms_3A_2.index)] = [i, x, y, z, d]

  if Atoms_3A_2.shape[0]>=Atoms_3A_1.shape[0]:
    Atoms_3A = Atoms_3A_2
  else:
    Atoms_3A = Atoms_3A_1

  if Atoms_3A.shape[0] < 2:
    with open("drive/MyDrive/Dataset/Diff_struct_new.csv", mode="a", encoding='utf-8') as w_file:
      file_writer = csv.writer(w_file, delimiter = ",")
      file_writer.writerow([str_num, struct_id])
    continue
  dots_max_dist = maxDist(Atoms_3A[['x', 'y', 'z']])

  #вектор, соединяющий точки с наибольшим расстоянием
  vec_3 = [[Atoms_3A['x'][dots_max_dist[0]] - Atoms_3A['x'][dots_max_dist[1]], Atoms_3A['y'][dots_max_dist[0]] - Atoms_3A['y'][dots_max_dist[1]], Atoms_3A['z'][dots_max_dist[0]] - Atoms_3A['z'][dots_max_dist[1]]]]

  # Получение проекции вектора vec_5 на разделяющую плоскость
  # vector u 
  u = np.array(vec_3)   
  
  # vector n: n is orthogonal vector to Plane P
  n = np.array([plane_bind[0], plane_bind[1], plane_bind[2]])       
   
  # Task: Project vector u on Plane P
  
  # finding norm of the vector n 
  n_norm = np.sqrt(sum(n**2))    
   
  # Apply the formula as mentioned above
  # for projecting a vector onto the orthogonal vector n
  # find dot product using np.dot()
  proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
  proj_vec = u - proj_of_u_on_n
  proj_vec = np.reshape(proj_vec, -1)

  norm = plane_bind[:3]
  a = math.sqrt(1/sum([coor**2 for coor in norm]))
  norm = [a*coor for coor in norm]
  norm = np.array(norm)
  a = math.sqrt(1/sum([coor**2 for coor in proj_vec]))
  proj_vec = a*proj_vec
  last_vector = np.cross(norm, proj_vec)

  #Создаю копию таблицы с атомами
  Atoms_coord = Atoms_coord_df.copy()
  Atoms_coord['x'] = Atoms_coord['x']-dot_is[0]
  Atoms_coord['y'] = Atoms_coord['y']-dot_is[1]
  Atoms_coord['z'] = Atoms_coord['z']-dot_is[2]

  from_bazis = np.concatenate((np.expand_dims(proj_vec, axis=1),
                             np.expand_dims(last_vector, axis=1), 
                             np.expand_dims(norm, axis=1)
                            ), axis=1)
  to_bazis = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
  s = np.matmul(np.linalg.inv(from_bazis), to_bazis)
  x_new = []
  y_new = []
  z_new = []
  for i, item in Atoms_coord.iterrows():
      vec = np.array([[item['x']], [item['y']], [item['z']]])
      new_vec = np.matmul(s, vec) 
      new_xy = rotate_point_wrt_center(point_to_be_rotated = (new_vec[0][0], new_vec[1][0]), angle = -45, center_point = (0, 0))
      x_new.append(new_xy[0])
      y_new.append(new_xy[1])
      z_new.append(new_vec[2][0])
  Atoms_coord['x_new'] = x_new
  Atoms_coord['y_new'] = y_new
  Atoms_coord['z_new'] = z_new
  #logreg_clf = LogisticRegression()
  #logreg_clf.fit(Atoms_coord[['x_new', 'y_new', 'z_new']].values, Atoms_coord_df['Chain_num'].astype('int'))

  # plane_bind - список коэффициентов [A,B,C,D] в уравнении разделяющей плоскости вида A*x+B*y+C*z+D=0, то есть 
  #plane_bind = list(logreg_clf.coef_[0])
  #plane_bind.append(logreg_clf.intercept_[0])
  high_gr = 10
  length_gr = 30
  width_gr = 30
  high_v = [[0, 0, -high_gr], [0, 0, high_gr]]
  length_v = [[-length_gr, 0, 0], [length_gr, 0, 0]]
  width_v = [[0, -width_gr, 0], [0, width_gr, 0]]


  #fig = plt.figure(figsize=(16, 10))
  #ax = plt.axes(projection="3d")
  #ax.plot3D([high_v[0][0],high_v[1][0]], [high_v[0][1],high_v[1][1]], [high_v[0][2],high_v[1][2]], 'red', linewidth= 5)
  #ax.plot3D([length_v[0][0],length_v[1][0]], [length_v[0][1],length_v[1][1]], [length_v[0][2],length_v[1][2]], 'green', linewidth= 5)
  #ax.plot3D([width_v[0][0],width_v[1][0]], [width_v[0][1],width_v[1][1]], [width_v[0][2],width_v[1][2]], 'blue', linewidth= 5)
  #ax.scatter3D(Atoms_coord['x_new'], Atoms_coord['y_new'], Atoms_coord['z_new'], c = Atoms_coord['Chain_num'])
  #plot_linear_cube(-length_gr, -width_gr, -high_gr, length_gr*2, width_gr*2, high_gr*2, color='gray')

  #ax.view_init(3, 90)

  #plt.savefig(f'drive/MyDrive/Cell_images/{struct_id}_0.png')
  #plt.close(fig)

  Cell_atoms = pd.DataFrame(columns=['Chain_name', 'Chain_num', 'Residue', 'Type', 'x_new', 'y_new', 'z_new'])
  for id_atom in range(Atoms_coord.shape[0]):
    at_coord = Atoms_coord.iloc[id_atom]
    if abs(at_coord['x_new'])<= length_gr and abs(at_coord['y_new'])<= width_gr and abs(at_coord['z_new'])<= high_gr:
      x_ind = round(at_coord['x_new']+length_gr)
      y_ind = round(at_coord['y_new']+width_gr)
      z_ind = round(at_coord['z_new']+high_gr)
      Cell_atoms.loc[len(Cell_atoms.index)] = [at_coord['Chain_name'], at_coord['Chain_num'], at_coord['Residue'], at_coord['Type'], x_ind, y_ind, z_ind]
  if Cell_atoms.shape[0] < 10:
    with open("drive/MyDrive/Dataset/Diff_struct_new.csv", mode="a", encoding='utf-8') as w_file:
      file_writer = csv.writer(w_file, delimiter = ",")
      file_writer.writerow([str_num, struct_id])
    continue
  Cell_atoms.to_csv(f'drive/MyDrive/Cells_tables/{struct_id}_neg45.csv')
  hb_a = []
  hb_d = []
  w_hb_d = []
  pos_charge = []
  neg_charge = []
  hph = []
  carb_o = []
  carb_c = []
  ar = []
  for i in range(Cell_atoms.shape[0]):
    atom_str = Cell_atoms.iloc[i]
    if atom_str['Chain_num']==0:
      arr_item = -1
    else:
      arr_item = 1
    if atom_str['Residue'] in hbond_acceptor and atom_str['Type'] in hbond_acceptor[atom_str['Residue']]:

      acceptor = arr_item
    else:
      acceptor = 0
    hb_a.append(acceptor)  

    if atom_str['Residue'] in hbond_donor and atom_str['Type'] in hbond_donor[atom_str['Residue']]:
      donor = arr_item
    else:
      donor = 0
    hb_d.append(donor) 

    if atom_str['Residue'] in weak_hbond_donor and atom_str['Type'] in weak_hbond_donor[atom_str['Residue']]:
      wdonor = arr_item
    else:
      wdonor = 0
    w_hb_d.append(wdonor)

    if atom_str['Residue'] in pos_ionisable and atom_str['Type'] in pos_ionisable[atom_str['Residue']]:
      pos = arr_item
    else:
      pos = 0
    pos_charge.append(pos)

    if atom_str['Residue'] in neg_ionisable and atom_str['Type'] in neg_ionisable[atom_str['Residue']]:
      neg = arr_item
    else:
      neg = 0
    neg_charge.append(neg)

    if atom_str['Residue'] in hydrophobe and atom_str['Type'] in hydrophobe[atom_str['Residue']]:
      h = arr_item
    else:
      h = 0
    hph.append(h)

    if atom_str['Type'] == 'O':
      o = arr_item
    else:
      o = 0
    carb_o.append(o)

    if atom_str['Type'] == 'C':
      c = arr_item
    else:
      c = 0
    carb_c.append(c)

    if atom_str['Residue'] in aromatic and atom_str['Type'] in aromatic[atom_str['Residue']]:
      a = arr_item
    else:
      a = 0
    ar.append(a)

  Cell_atoms['HBond_acceptor'] = hb_a
  Cell_atoms['HBond_donor'] = hb_d
  Cell_atoms['Weak_HBond_donor'] = w_hb_d
  Cell_atoms['Pos_charge'] = pos_charge
  Cell_atoms['Neg_charge'] = neg_charge
  Cell_atoms['Hydrophoby'] = hph
  Cell_atoms['Carbocsile_o'] = carb_o
  Cell_atoms['Carbocsile_c'] = carb_c
  Cell_atoms['Aromatic'] = ar

  num_channels = 10
  coord_array = np.zeros((num_channels, length_gr*2+1, width_gr*2+1, high_gr*2+1))
  for a in range(Cell_atoms.shape[0]):
    x = Cell_atoms['x_new'][a]
    y = Cell_atoms['y_new'][a]
    z = Cell_atoms['z_new'][a]
    if Cell_atoms['Chain_num'][a] == 0:
      coord_array[0][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[1][x][y][z] = Cell_atoms['HBond_donor'][a]
      coord_array[2][x][y][z] = Cell_atoms['Weak_HBond_donor'][a]
      coord_array[3][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[4][x][y][z] = Cell_atoms['Pos_charge'][a]
      coord_array[5][x][y][z] = Cell_atoms['Neg_charge'][a]
      coord_array[6][x][y][z] = Cell_atoms['Hydrophoby'][a]
      coord_array[7][x][y][z] = Cell_atoms['Carbocsile_o'][a]
      coord_array[8][x][y][z] = Cell_atoms['Carbocsile_c'][a]
      coord_array[9][x][y][z] = Cell_atoms['Aromatic'][a]
    else:
      coord_array[0][x][y][z] = Cell_atoms['HBond_donor'][a]
      coord_array[1][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[2][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[3][x][y][z] = Cell_atoms['Weak_HBond_donor'][a]
      coord_array[4][x][y][z] = Cell_atoms['Neg_charge'][a]
      coord_array[5][x][y][z] = Cell_atoms['Pos_charge'][a]
      coord_array[6][x][y][z] = Cell_atoms['Hydrophoby'][a]
      coord_array[7][x][y][z] = Cell_atoms['Carbocsile_o'][a]
      coord_array[8][x][y][z] = Cell_atoms['Carbocsile_c'][a]
      coord_array[9][x][y][z] = Cell_atoms['Aromatic'][a]

  pKD = float(structures_files[2][str_num])
  with open(f'drive/MyDrive/Dataset/{struct_id}_neg45.npy', 'wb') as f:
    np.save(f, coord_array)
  
  with open("drive/MyDrive/Dataset/Data_files_neg45.csv", mode="a", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter = ",")
    file_writer.writerow([str_num, f'{struct_id}_neg45.npy', pKD])


In [ ]:
struct_many_chains = pd.read_csv('/content/drive/MyDrive/Complexes_chains.csv', index_col=0, header=0)
struct_many_chains

,pdb_id,number_chain,protein_1,protein_2,Difficulty
Num,,,,,
0,5bxq,5,A B,D C,0
1,1a3b,3,H,I,0
2,1a4y,4,D,E,0
3,1ahw,3,A D E,F,1
4,1ak4,4,A B,C D,0
...,...,...,...,...,...
1909,6uda,12,K L,I,0
1910,6um5,12,H G,E,0
1911,6uvo,3,H L,D,0


In [ ]:
list(structures_files[0][876:])

In [ ]:
structures_files[0][2478]

1722

In [ ]:
for str_num in list(structures_files[0][2496:]):
#for str_num in range(389, 601):
  if struct_many_chains['Difficulty'][str_num] == 0:
    struct_id = struct_many_chains['pdb_id'][str_num]
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(struct_id, pdir='drive/MyDrive/Structures')
    parser = MMCIFParser()
    pdb = f'drive/MyDrive/Structures/{struct_id}.cif'
    structure = parser.get_structure({struct_id}, pdb)
    clear_output()

    Atoms_coord_df = pd.DataFrame(columns=['Chain_name', 'Chain_num', 'Residue', 'Type', 'x', 'y', 'z']) 
    chains = structure[0].get_chains()
    for chain in chains:
      ch_id = chain.get_id()
      if ch_id not in struct_many_chains['protein_1'][str_num] and ch_id not in struct_many_chains['protein_2'][str_num]:
        continue
      else:
        if ch_id in struct_many_chains['protein_1'][str_num]:
          num_chain=0
        elif ch_id in struct_many_chains['protein_2'][str_num]:
          num_chain=1
        atoms = chain.get_atoms()
        for atom in atoms:
          coord_atom = atom.get_coord()
          name_atom = atom.get_name()
          if name_atom[0] == 'H':
            continue
          res_name = str(atom.get_parent())[9:12]
          if atom.get_full_id()[3][0] == ' ' and res_name!= 'UNK':
            Atoms_coord_df.loc[len(Atoms_coord_df.index)] = [ch_id, num_chain, res_name, name_atom, coord_atom[0], coord_atom[1], coord_atom[2]]
  else:
    continue
  #Atoms_coord_df.replace('OXT', 'O', inplace=True)

  #Получение центров масс для белков
  center_1=[]
  x1=0
  y1=0
  z1=0
  k1=0
  center_2=[]
  x2=0
  y2=0
  z2=0
  k2=0
  for j in range(Atoms_coord_df.shape[0]):
    if Atoms_coord_df['Chain_num'][j]==0:
      x1+=Atoms_coord_df['x'][j]
      y1+=Atoms_coord_df['y'][j]
      z1+=Atoms_coord_df['z'][j]
      k1+=1
    elif Atoms_coord_df['Chain_num'][j]==1:
      x2+=Atoms_coord_df['x'][j]
      y2+=Atoms_coord_df['y'][j]
      z2+=Atoms_coord_df['z'][j]
      k2+=1
  center_1=[round(x1/k1), round(y1/k1), round(z1/k1)]
  center_2=[round(x2/k2), round(y2/k2), round(z2/k2)]



  logreg_clf = LogisticRegression()
  logreg_clf.fit(Atoms_coord_df[['x', 'y', 'z']].values, Atoms_coord_df['Chain_num'].astype('int'))

  # plane_bind - список коэффициентов [A,B,C,D] в уравнении разделяющей плоскости вида A*x+B*y+C*z+D=0, то есть 
  plane_bind = list(logreg_clf.coef_[0])
  plane_bind.append(logreg_clf.intercept_[0])

  #dot_is - координаты точки пересечения, а следовательно и центра ячейки
  dot_is = isect_line_plane_v3_4d(center_1, center_2, plane_bind)
  x0, y0, z0 = dot_is[0], dot_is[1], dot_is[2]
  x1, y1, z1 = center_1[0], center_1[1], center_1[2]
  x2, y2, z2 = center_2[0], center_2[1], center_2[2]
  n_v = [(x1 - x0), (y1 - y0), (z1 - z0)]

  plane_3_1=[plane_bind[0], plane_bind[1], plane_bind[2], plane_bind[3]+3]
  plane_3_2=[plane_bind[0], plane_bind[1], plane_bind[2], plane_bind[3]-3]

  #Выбираем атомы, которые примерно лежат на этой плоскости, подставляем их координаты в уравнение плоскости, чтобы понять насколько близко он к ней расположены
  Atoms_3A_1 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
  for i in range(Atoms_coord_df.shape[0]):
    x = Atoms_coord_df['x'][i]
    y = Atoms_coord_df['y'][i]
    z = Atoms_coord_df['z'][i]
    d = plane_3_1[0]*x + plane_3_1[1]*y + plane_3_1[2]*z + plane_3_1[3]
    if abs(d)<0.3:
      Atoms_3A_1.loc[len(Atoms_3A_1.index)] = [i, x, y, z, d]

  Atoms_3A_2 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
  for i in range(Atoms_coord_df.shape[0]):
    x = Atoms_coord_df['x'][i]
    y = Atoms_coord_df['y'][i]
    z = Atoms_coord_df['z'][i]
    d = plane_3_2[0]*x + plane_3_2[1]*y + plane_3_2[2]*z + plane_3_2[3]
    if abs(d)<0.3:
      Atoms_3A_2.loc[len(Atoms_3A_2.index)] = [i, x, y, z, d]

  if Atoms_3A_2.shape[0]>=Atoms_3A_1.shape[0]:
    Atoms_3A = Atoms_3A_2
  else:
    Atoms_3A = Atoms_3A_1

  if Atoms_3A.shape[0] < 2:
    with open("drive/MyDrive/Dataset/Diff_struct_new.csv", mode="a", encoding='utf-8') as w_file:
      file_writer = csv.writer(w_file, delimiter = ",")
      file_writer.writerow([str_num, struct_id])
    continue
  dots_max_dist = maxDist(Atoms_3A[['x', 'y', 'z']])

  #вектор, соединяющий точки с наибольшим расстоянием
  vec_3 = [[Atoms_3A['x'][dots_max_dist[0]] - Atoms_3A['x'][dots_max_dist[1]], Atoms_3A['y'][dots_max_dist[0]] - Atoms_3A['y'][dots_max_dist[1]], Atoms_3A['z'][dots_max_dist[0]] - Atoms_3A['z'][dots_max_dist[1]]]]

  # Получение проекции вектора vec_5 на разделяющую плоскость
  # vector u 
  u = np.array(vec_3)   
  
  # vector n: n is orthogonal vector to Plane P
  n = np.array([plane_bind[0], plane_bind[1], plane_bind[2]])       
   
  # Task: Project vector u on Plane P
  
  # finding norm of the vector n 
  n_norm = np.sqrt(sum(n**2))    
   
  # Apply the formula as mentioned above
  # for projecting a vector onto the orthogonal vector n
  # find dot product using np.dot()
  proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
  proj_vec = u - proj_of_u_on_n
  proj_vec = np.reshape(proj_vec, -1)

  norm = plane_bind[:3]
  a = math.sqrt(1/sum([coor**2 for coor in norm]))
  norm = [a*coor for coor in norm]
  norm = np.array(norm)
  a = math.sqrt(1/sum([coor**2 for coor in proj_vec]))
  proj_vec = a*proj_vec
  last_vector = np.cross(norm, proj_vec)

  #Создаю копию таблицы с атомами
  Atoms_coord = Atoms_coord_df.copy()
  Atoms_coord['x'] = Atoms_coord['x']-dot_is[0]
  Atoms_coord['y'] = Atoms_coord['y']-dot_is[1]
  Atoms_coord['z'] = Atoms_coord['z']-dot_is[2]

  from_bazis = np.concatenate((np.expand_dims(proj_vec, axis=1),
                             np.expand_dims(last_vector, axis=1), 
                             np.expand_dims(norm, axis=1)
                            ), axis=1)
  to_bazis = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
  s = np.matmul(np.linalg.inv(from_bazis), to_bazis)
  x_new = []
  y_new = []
  z_new = []
  for i, item in Atoms_coord.iterrows():
      vec = np.array([[item['x']], [item['y']], [item['z']]])
      new_vec = np.matmul(s, vec)
      new_xy = rotate_point_wrt_center(point_to_be_rotated = (new_vec[0][0], new_vec[1][0]), angle = -45, center_point = (0, 0))
      x_new.append(new_xy[0])
      y_new.append(new_xy[1]) 
      z_new.append(new_vec[2][0])
  Atoms_coord['x_new'] = x_new
  Atoms_coord['y_new'] = y_new
  Atoms_coord['z_new'] = z_new
  #logreg_clf = LogisticRegression()
  #logreg_clf.fit(Atoms_coord[['x_new', 'y_new', 'z_new']].values, Atoms_coord_df['Chain_num'].astype('int'))

  # plane_bind - список коэффициентов [A,B,C,D] в уравнении разделяющей плоскости вида A*x+B*y+C*z+D=0, то есть 
  #plane_bind = list(logreg_clf.coef_[0])
  #plane_bind.append(logreg_clf.intercept_[0])
  high_gr = 10
  length_gr = 30
  width_gr = 30
  high_v = [[0, 0, -high_gr], [0, 0, high_gr]]
  length_v = [[-length_gr, 0, 0], [length_gr, 0, 0]]
  width_v = [[0, -width_gr, 0], [0, width_gr, 0]]


  #fig = plt.figure(figsize=(16, 10))
  #ax = plt.axes(projection="3d")
  #ax.plot3D([high_v[0][0],high_v[1][0]], [high_v[0][1],high_v[1][1]], [high_v[0][2],high_v[1][2]], 'red', linewidth= 5)
  #ax.plot3D([length_v[0][0],length_v[1][0]], [length_v[0][1],length_v[1][1]], [length_v[0][2],length_v[1][2]], 'green', linewidth= 5)
  #ax.plot3D([width_v[0][0],width_v[1][0]], [width_v[0][1],width_v[1][1]], [width_v[0][2],width_v[1][2]], 'blue', linewidth= 5)
  #ax.scatter3D(Atoms_coord['x_new'], Atoms_coord['y_new'], Atoms_coord['z_new'], c = Atoms_coord['Chain_num'])
  #plot_linear_cube(-length_gr, -width_gr, -high_gr, length_gr*2, width_gr*2, high_gr*2, color='gray')

  #ax.view_init(3, 90)

  #plt.savefig(f'drive/MyDrive/Cell_images/{struct_id}.png')
  #plt.close(fig)

  Cell_atoms = pd.DataFrame(columns=['Chain_name', 'Chain_num', 'Residue', 'Type', 'x_new', 'y_new', 'z_new'])
  for id_atom in range(Atoms_coord.shape[0]):
    at_coord = Atoms_coord.iloc[id_atom]
    if abs(at_coord['x_new'])<= length_gr and abs(at_coord['y_new'])<= width_gr and abs(at_coord['z_new'])<= high_gr:
      x_ind = round(at_coord['x_new']+length_gr)
      y_ind = round(at_coord['y_new']+width_gr)
      z_ind = round(at_coord['z_new']+high_gr)
      Cell_atoms.loc[len(Cell_atoms.index)] = [at_coord['Chain_name'], at_coord['Chain_num'], at_coord['Residue'], at_coord['Type'], x_ind, y_ind, z_ind]
  if Cell_atoms.shape[0] < 10:
    with open("drive/MyDrive/Dataset/Diff_struct_new.csv", mode="a", encoding='utf-8') as w_file:
      file_writer = csv.writer(w_file, delimiter = ",")
      file_writer.writerow([str_num, struct_id])
    continue
  Cell_atoms.to_csv(f'drive/MyDrive/Cells_tables/{struct_id}_neg45.csv')
  hb_a = []
  hb_d = []
  w_hb_d = []
  pos_charge = []
  neg_charge = []
  hph = []
  carb_o = []
  carb_c = []
  ar = []
  for i in range(Cell_atoms.shape[0]):
    atom_str = Cell_atoms.iloc[i]
    if atom_str['Chain_num']==0:
      arr_item = -1
    else:
      arr_item = 1
    if atom_str['Residue'] in hbond_acceptor and atom_str['Type'] in hbond_acceptor[atom_str['Residue']]:

      acceptor = arr_item
    else:
      acceptor = 0
    hb_a.append(acceptor)  

    if atom_str['Residue'] in hbond_donor and atom_str['Type'] in hbond_donor[atom_str['Residue']]:
      donor = arr_item
    else:
      donor = 0
    hb_d.append(donor) 

    if atom_str['Residue'] in weak_hbond_donor and atom_str['Type'] in weak_hbond_donor[atom_str['Residue']]:
      wdonor = arr_item
    else:
      wdonor = 0
    w_hb_d.append(wdonor)

    if atom_str['Residue'] in pos_ionisable and atom_str['Type'] in pos_ionisable[atom_str['Residue']]:
      pos = arr_item
    else:
      pos = 0
    pos_charge.append(pos)

    if atom_str['Residue'] in neg_ionisable and atom_str['Type'] in neg_ionisable[atom_str['Residue']]:
      neg = arr_item
    else:
      neg = 0
    neg_charge.append(neg)

    if atom_str['Residue'] in hydrophobe and atom_str['Type'] in hydrophobe[atom_str['Residue']]:
      h = arr_item
    else:
      h = 0
    hph.append(h)

    if atom_str['Type'] == 'O':
      o = arr_item
    else:
      o = 0
    carb_o.append(o)

    if atom_str['Type'] == 'C':
      c = arr_item
    else:
      c = 0
    carb_c.append(c)

    if atom_str['Residue'] in aromatic and atom_str['Type'] in aromatic[atom_str['Residue']]:
      a = arr_item
    else:
      a = 0
    ar.append(a)

  Cell_atoms['HBond_acceptor'] = hb_a
  Cell_atoms['HBond_donor'] = hb_d
  Cell_atoms['Weak_HBond_donor'] = w_hb_d
  Cell_atoms['Pos_charge'] = pos_charge
  Cell_atoms['Neg_charge'] = neg_charge
  Cell_atoms['Hydrophoby'] = hph
  Cell_atoms['Carbocsile_o'] = carb_o
  Cell_atoms['Carbocsile_c'] = carb_c
  Cell_atoms['Aromatic'] = ar

  num_channels = 10
  coord_array = np.zeros((num_channels, length_gr*2+1, width_gr*2+1, high_gr*2+1))
  for a in range(Cell_atoms.shape[0]):
    x = Cell_atoms['x_new'][a]
    y = Cell_atoms['y_new'][a]
    z = Cell_atoms['z_new'][a]
    if Cell_atoms['Chain_num'][a] == 0:
      coord_array[0][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[1][x][y][z] = Cell_atoms['HBond_donor'][a]
      coord_array[2][x][y][z] = Cell_atoms['Weak_HBond_donor'][a]
      coord_array[3][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[4][x][y][z] = Cell_atoms['Pos_charge'][a]
      coord_array[5][x][y][z] = Cell_atoms['Neg_charge'][a]
      coord_array[6][x][y][z] = Cell_atoms['Hydrophoby'][a]
      coord_array[7][x][y][z] = Cell_atoms['Carbocsile_o'][a]
      coord_array[8][x][y][z] = Cell_atoms['Carbocsile_c'][a]
      coord_array[9][x][y][z] = Cell_atoms['Aromatic'][a]
    else:
      coord_array[0][x][y][z] = Cell_atoms['HBond_donor'][a]
      coord_array[1][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[2][x][y][z] = Cell_atoms['HBond_acceptor'][a]
      coord_array[3][x][y][z] = Cell_atoms['Weak_HBond_donor'][a]
      coord_array[4][x][y][z] = Cell_atoms['Neg_charge'][a]
      coord_array[5][x][y][z] = Cell_atoms['Pos_charge'][a]
      coord_array[6][x][y][z] = Cell_atoms['Hydrophoby'][a]
      coord_array[7][x][y][z] = Cell_atoms['Carbocsile_o'][a]
      coord_array[8][x][y][z] = Cell_atoms['Carbocsile_c'][a]
      coord_array[9][x][y][z] = Cell_atoms['Aromatic'][a]

  str_pKD = df_all_complexes[df_all_complexes['PDB code'] == struct_id]
  pKD = float(str_pKD['pKd pKi pIC50'])
  with open(f'drive/MyDrive/Dataset/{struct_id}_neg45.npy', 'wb') as f:
    np.save(f, coord_array)
  
  with open("drive/MyDrive/Dataset/Data_files_neg45.csv", mode="a", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter = ",")
    file_writer.writerow([str_num, f'{struct_id}_neg45.npy', pKD])

In [ ]:
str_num

1673

In [ ]:
struct_id

'6bxa'